In [1]:
import cantera as ct
import numpy as np
import matplotlib.pyplot as plt
#Atmosphere
H=11000 # Wysokość w m
M=0.5 # Mach Number
Tbaz=15 # Base Temperature i Celcius degrees
#Calculatins
p0=101325*(1-H/44300)**5.256
ro0=1.2255*(1-H/44300)**4.256
if H>11000:
    T0=(Tbaz+273.15)-6.5*11000/1000
else:
    T0=(Tbaz+273.15)-6.5*H/1000 
R=286.9
k=1.4
a0=(1.4*286.9*T0)**0.5
V0=M*a0
# Atmospheric results
print('T0=',T0,'K')
print('p0=',p0,'Pa')
print('ro0=',ro0,'kg/m^3')
print('a0=',a0,'m/s')
print('V=',V0,'m/s')
#Engine propeties
# Intake
PId=0.98
fi_d=0.5 # intake diameter [m]
mdot_stat=100 #stationary mass flow
# FAN
BPR=1.5 #Bypass Ratio
PIf=3.5
SPRf=0.97
# compressor
PIc=15
SPRc=0.92
vc=0.02
# turbine
SPRt=0.95
SPRmech=0.99
# nozzle
fi_n=0.95
# CC
PIKS=0.98
f=0.026 # Normalnie jest około 0.003 ale spalanie C3H8 wymaga większej ilości paliwa w silniku turbinowym ***
# CORE
f=np.arange(0,0.04,0.001)
Tt4=np.arange(0,0.04,0.001)
Thrust=np.arange(0,0.04,0.001)
Dana11=np.arange(0,0.04,0.001)
Dana12=np.arange(0,0.04,0.001)
Dana13=np.arange(0,0.04,0.001)
Dana14=np.arange(0,0.04,0.001)
i=0
for i in range(len(f)):
    mdot=(mdot_stat+(3.14*fi_d**2/4)*V0)*ro0
    # INTAKE - MAKING GAS
    GAZ_1=ct.Solution('gri30.xml')
    GAZ_1.TPX=T0,p0,'O2:0.21, N2:0.78, AR:0.01'
    ro_1=GAZ_1.density
    GAZ_1()
    print(mdot,'kg/s')
    # Intake equasions
    GAZ_1.TP=GAZ_1.T*(1+((k-1)/2)*M**2),GAZ_1.P*((1+((k-1)/2)*M**2)**(k/(k-1)))*PId
    GAZ_1()
    # FAN - BPR
    Lf=(GAZ_1.T*R*(k/(k-1))*(PIf**((k-1)/k)-1))/SPRf # Wymagana praca
    GAZ_1.TP=GAZ_1.T+(Lf*(k-1))/(k*R),GAZ_1.P*PIf
    # Splitting in 2 streams
    mdotA=mdot*BPR/(BPR+1)
    mdotB=mdot-mdotA
    print(Lf)
    print(mdotA,mdotB)
    GAZ_A=ct.Solution('gri30.xml')
    GAZ_B=ct.Solution('gri30.xml')
    GAZ_A.TPX=GAZ_1.T,GAZ_1.P,GAZ_1.X
    GAZ_B.TPX=GAZ_1.T,GAZ_1.P,GAZ_1.X
    GAZ_A()
    #Compressor
    # Gas B through compressor
    Lc=(GAZ_B.T*R*(k/(k-1))*(PIc**((k-1)/k)-1))/SPRc
    print(Lc)
    GAZ_B.TP=GAZ_B.T+(Lc*(k-1)/(k*R)),GAZ_B.P*PIc
    GAZ_B()
    #CC - fuel stream injection
    mdotF=mdotB*f[i]
    #Making fuel stream
    GAZ_F=ct.Solution('gri30.xml')
    GAZ_F.TPX=273.15+15,GAZ_B.P,'C3H8:1' 
    #Combining two streams B and F
    ro_B=GAZ_B.density 
    ro_F=GAZ_F.density 
    res_B=ct.Reservoir(GAZ_B) 
    res_F=ct.Reservoir(GAZ_F) 
    downstream=ct.Reservoir(GAZ_F) 
    GAZ_C=ct.Solution('gri30.xml')
    GAZ_C.TPX=GAZ_B.T,GAZ_B.P,GAZ_B.X 
    mixer = ct.IdealGasReactor(GAZ_C) 
    mfc1 = ct.MassFlowController(res_B, mixer, mdot=mdotB) 
    mfc2 = ct.MassFlowController(res_F, mixer, mdot=mdotF)
    outlet = ct.Valve(mixer, downstream, K=10.0)
    sim = ct.ReactorNet([mixer])
    sim.advance_to_steady_state()
    print(mixer.thermo.report())
    # Combustion "C"
    # Making C
    mdotC=mdotB+mdotF
    # Reaction gas C
    inlet = ct.Reservoir(GAZ_C)
    GAZ_C.equilibrate('HP')
    combustor = ct.IdealGasReactor(GAZ_C)
    combustor.volume = 500.0
    exhaust = ct.Reservoir(GAZ_C)
    inlet_mfc = ct.MassFlowController(inlet, combustor, mdot=mdotC)
    outlet_mfc = ct.PressureController(combustor, exhaust, master=inlet_mfc, K=0.01)
    sim = ct.ReactorNet([combustor])
    sim.advance_to_steady_state()
    print(combustor.thermo.report())
    # CC Exit
    GAZ_C.TP=GAZ_C.T,GAZ_C.P*PIKS
    Tt4[i]=GAZ_C.T
    # Gas C throught Turbine
    Lt=(Lc+Lf*BPR)/((1+f[i])*(1-vc)*SPRmech)
    print(Lt)
    Rx=GAZ_C.cp-GAZ_C.cv
    kx=GAZ_C.cp/GAZ_C.cv
    print("R1=",Rx)
    print("k1=",kx)
    PIt=(1-((Lt*(kx-1))/(Rx*GAZ_C.T*kx*SPRt)))**(kx/(kx-1))
    print ("PIt=",PIt)
    GAZ_C.TP=GAZ_C.T-(Lt*(kx-1))/(kx*Rx),GAZ_C.P*PIt
    GAZ_C()
    # Mixing two streams A and C
    # cross section 16
    GAZ_A.TP=GAZ_A.T,GAZ_A.P*0.99
    # Mixing A i C = Gas D
    ro_A=GAZ_A.density 
    ro_C=GAZ_C.density 
    res_A=ct.Reservoir(GAZ_A) 
    res_C=ct.Reservoir(GAZ_C)  
    GAZ_D=ct.Solution('gri30.xml')
    GAZ_D.TPX=GAZ_A.T,(GAZ_A.P+GAZ_C.P)/2,GAZ_A.X 
    downstream=ct.Reservoir(GAZ_D)
    mixer = ct.IdealGasReactor(GAZ_D) 
    mfc1 = ct.MassFlowController(res_A, mixer, mdot=mdotA) 
    mfc2 = ct.MassFlowController(res_C, mixer, mdot=mdotC)
    outlet = ct.Valve(mixer, downstream, K=10.0)
    sim = ct.ReactorNet([mixer])
    sim.advance_to_steady_state()
    #print(mixer.thermo.report())
    GAZ_D()
    RD=GAZ_D.cp-GAZ_D.cv
    kD=GAZ_D.cp/GAZ_D.cv
    print(RD,kD)
    mdotD=mdotA+mdotC
    # Nozzle
    # 3 cases
    PI_n=p0/GAZ_D.P
    PI_cr=(2/(kD+1))**(kD/(kD-1))
    print(PI_n,PI_cr)
    if PI_n<=0.2:
        V9=fi_n*(2*kD*RD*GAZ_D.T*(1-PI_n**((kD-1)/kD))/(kD-1))**0.5
        GAZ_D.TP=GAZ_D.T-(V9**2)*(kD-1)/(2*kD*RD),p0
    else:
        if PI_cr>PI_n:
            V9=fi_n*(2*kD*RD*GAZ_D.T/(kD+1))**0.5
            GAZ_D.TP=GAZ_D.T-(V9**2)*(kD-1)/(2*kD*RD),GAZ_D.P*PI_cr
        else:
            V9=fi_n*(2*kD*RD*GAZ_D.T*(1-PI_n**((kD-1)/kD))/(kD-1))**0.5
            GAZ_D.TP=GAZ_D.T-(V9**2)*(kD-1)/(2*kD*RD),p0
    print(V9,GAZ_D.T,GAZ_D.P)
    # REsults
    kj=(1+f[i])*V9-V0+(1+f[i])*(GAZ_D.P-p0)/(GAZ_D.density*V9)
    Dana11[i]=V9
    Thrust[i]=kj*mdot
    if Thrust[i]<0:
        Thrust[i]=0
        Dana12[i]=0
        Dana13[i]=100
        Dana14[i]=Dana12[i]*Dana13[i]
    else:
        #Efficiency
        Vzrzr=kj+V0
        Dana12[i]=(Vzrzr**2-V0**2)/(46000000*2*f[i])*100
        Dana13[i]=2/(1+(Vzrzr/V0))*100
        Dana14[i]=Dana12[i]*Dana13[i]/100

###############    New set #############
fxx=0.02
M=np.arange(0,1,0.1)
Dana21=np.arange(0,1,0.1)
Dana22=np.arange(0,1,0.1)
Dana23=np.arange(0,1,0.1)
Dana24=np.arange(0,1,0.1)
Dana25=np.arange(0,1,0.1)
Dana26=np.arange(0,1,0.1)
Dana30=np.arange(0,1,0.1)

i=0
for i in range(len(M)):
    V0=M[i]*a0
    mdot=(mdot_stat+(3.14*fi_d**2/4)*V0)*ro0
    # INTAKE - MAKING GAS
    GAZ_1=ct.Solution('gri30.xml')
    GAZ_1.TPX=T0,p0,'O2:0.21, N2:0.78, AR:0.01'
    ro_1=GAZ_1.density
    GAZ_1()
    print(mdot,'kg/s')
    # Intake equasions
    GAZ_1.TP=GAZ_1.T*(1+((k-1)/2)*M[i]**2),GAZ_1.P*((1+((k-1)/2)*M[i]**2)**(k/(k-1)))*PId
    GAZ_1()
    # FAN - BPR
    Lf=(GAZ_1.T*R*(k/(k-1))*(PIf**((k-1)/k)-1))/SPRf # Wymagana praca
    GAZ_1.TP=GAZ_1.T+(Lf*(k-1))/(k*R),GAZ_1.P*PIf
    # Splitting in 2 streams
    mdotA=mdot*BPR/(BPR+1)
    mdotB=mdot-mdotA
    print(Lf)
    print(mdotA,mdotB)
    GAZ_A=ct.Solution('gri30.xml')
    GAZ_B=ct.Solution('gri30.xml')
    GAZ_A.TPX=GAZ_1.T,GAZ_1.P,GAZ_1.X
    GAZ_B.TPX=GAZ_1.T,GAZ_1.P,GAZ_1.X
    GAZ_A()
    #Compressor
    # Gas B through compressor
    Lc=(GAZ_B.T*R*(k/(k-1))*(PIc**((k-1)/k)-1))/SPRc
    print(Lc)
    GAZ_B.TP=GAZ_B.T+(Lc*(k-1)/(k*R)),GAZ_B.P*PIc
    GAZ_B()
    #CC - fuel stream injection
    mdotF=mdotB*fxx
    #Making fuel stream
    GAZ_F=ct.Solution('gri30.xml')
    GAZ_F.TPX=273.15+15,GAZ_B.P,'C3H8:1' 
    #Combining two streams B and F
    ro_B=GAZ_B.density 
    ro_F=GAZ_F.density 
    res_B=ct.Reservoir(GAZ_B) 
    res_F=ct.Reservoir(GAZ_F) 
    downstream=ct.Reservoir(GAZ_F) 
    GAZ_C=ct.Solution('gri30.xml')
    GAZ_C.TPX=GAZ_B.T,GAZ_B.P,GAZ_B.X 
    mixer = ct.IdealGasReactor(GAZ_C) 
    mfc1 = ct.MassFlowController(res_B, mixer, mdot=mdotB) 
    mfc2 = ct.MassFlowController(res_F, mixer, mdot=mdotF)
    outlet = ct.Valve(mixer, downstream, K=10.0)
    sim = ct.ReactorNet([mixer])
    sim.advance_to_steady_state()
    print(mixer.thermo.report())
    # Combustion "C"
    # Making C
    mdotC=mdotB+mdotF
    # Reaction gas C
    inlet = ct.Reservoir(GAZ_C)
    GAZ_C.equilibrate('HP')
    combustor = ct.IdealGasReactor(GAZ_C)
    combustor.volume = 500.0
    exhaust = ct.Reservoir(GAZ_C)
    inlet_mfc = ct.MassFlowController(inlet, combustor, mdot=mdotC)
    outlet_mfc = ct.PressureController(combustor, exhaust, master=inlet_mfc, K=0.01)
    sim = ct.ReactorNet([combustor])
    sim.advance_to_steady_state()
    print(combustor.thermo.report())
    # CC Exit
    GAZ_C.TP=GAZ_C.T,GAZ_C.P*PIKS
    Dana21[i]=GAZ_C.T
    # Gas C throught Turbine
    Lt=(Lc+Lf*BPR)/((1+fxx)*(1-vc)*SPRmech)
    print(Lt)
    Rx=GAZ_C.cp-GAZ_C.cv
    kx=GAZ_C.cp/GAZ_C.cv
    print("R1=",Rx)
    print("k1=",kx)
    PIt=(1-((Lt*(kx-1))/(Rx*GAZ_C.T*kx*SPRt)))**(kx/(kx-1))
    print ("PIt=",PIt)
    GAZ_C.TP=GAZ_C.T-(Lt*(kx-1))/(kx*Rx),GAZ_C.P*PIt
    GAZ_C()
    # Mixing two streams A and C
    # cross section 16
    GAZ_A.TP=GAZ_A.T,GAZ_A.P*0.99
    # Mixing A i C = Gas D
    ro_A=GAZ_A.density 
    ro_C=GAZ_C.density 
    res_A=ct.Reservoir(GAZ_A) 
    res_C=ct.Reservoir(GAZ_C)  
    GAZ_D=ct.Solution('gri30.xml')
    GAZ_D.TPX=GAZ_A.T,(GAZ_A.P+GAZ_C.P)/2,GAZ_A.X 
    downstream=ct.Reservoir(GAZ_D)
    mixer = ct.IdealGasReactor(GAZ_D) 
    mfc1 = ct.MassFlowController(res_A, mixer, mdot=mdotA) 
    mfc2 = ct.MassFlowController(res_C, mixer, mdot=mdotC)
    outlet = ct.Valve(mixer, downstream, K=10.0)
    sim = ct.ReactorNet([mixer])
    sim.advance_to_steady_state()
    #print(mixer.thermo.report())
    GAZ_D()
    RD=GAZ_D.cp-GAZ_D.cv
    kD=GAZ_D.cp/GAZ_D.cv
    print(RD,kD)
    mdotD=mdotA+mdotC
    # Nozzle
    if i==5:
        GAZ_E=ct.Solution('gri30.xml')
        GAZ_E.TPX=GAZ_D.P,GAZ_D.P,GAZ_D.X
    
    # 3 cases
    PI_n=p0/GAZ_D.P
    PI_cr=(2/(kD+1))**(kD/(kD-1))
    print(PI_n,PI_cr)
    if PI_n<=0.2:
        V9=fi_n*(2*kD*RD*GAZ_D.T*(1-PI_n**((kD-1)/kD))/(kD-1))**0.5
        GAZ_D.TP=GAZ_D.T-(V9**2)*(kD-1)/(2*kD*RD),p0
    else:
        if PI_cr>PI_n:
            V9=fi_n*(2*kD*RD*GAZ_D.T/(kD+1))**0.5
            GAZ_D.TP=GAZ_D.T-(V9**2)*(kD-1)/(2*kD*RD),GAZ_D.P*PI_cr
        else:
            V9=fi_n*(2*kD*RD*GAZ_D.T*(1-PI_n**((kD-1)/kD))/(kD-1))**0.5
            GAZ_D.TP=GAZ_D.T-(V9**2)*(kD-1)/(2*kD*RD),p0
    print(V9,GAZ_D.T,GAZ_D.P)
    # REsults
    kj=(1+fxx)*V9-V0+(1+fxx)*(GAZ_D.P-p0)/(GAZ_D.density*V9)
    Dana23[i]=V9
    Dana22[i]=kj*mdot
    if Dana22[i]<0:
        Dana22[i]=0
        Dana24[i]=0
        Dana25[i]=100
        Dana26[i]=Dana24[i]*Dana25[i]
    else:
        #Efficiency
        Vzrzr=kj+V0
        Dana24[i]=(Vzrzr**2-V0**2)/(46000000*2*fxx)*100
        Dana25[i]=2/(1+(Vzrzr/V0))*100
        Dana26[i]=Dana24[i]*Dana25[i]/100
        

#Making REsults/plots
print('Tt4(f)[K]')
plt.plot(f,Tt4)
plt.show()
print('Thrust(f)[kN]')
plt.plot(f,Thrust/1000)
plt.show()
print('V9(f)[m/s]')
plt.plot(f,Dana11)
plt.show()
print('nt(f)[%]')
plt.plot(f,Dana12)
plt.show()
print('no(f)[%]')
plt.plot(f,Dana13)
plt.show()
print('n(f)[%]')
plt.plot(f,Dana14)
plt.show()
print('NEW SET')
print('Tt4(M)[K]')
plt.plot(M,Dana21)
plt.show()
print('Thrust(M)[kN]')
plt.plot(M,Dana22/1000)
plt.show()
print('V9(M)[m/s]')
plt.plot(M,Dana23)
plt.show()
print('nt(M)[%]')
plt.plot(M,Dana24)
plt.show()
print('no(M)[%]')
plt.plot(M,Dana25)
plt.show()
print('n(M)[%]')
plt.plot(M,Dana26)
plt.show()
GAZ_E()

T0= 216.64999999999998 K
p0= 22604.01145405448 Pa
ro0= 0.3636986714135627 kg/m^3
a0= 294.9909134193797 m/s
V= 147.49545670968985 m/s

  gri30:

       temperature          216.65  K
          pressure           22604  Pa
           density        0.363528  kg/m^3
  mean mol. weight         28.9697  amu

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy          -81371       -2.357e+06     J
   internal energy     -1.4355e+05       -4.159e+06     J
           entropy          6971.5         2.02e+05     J/K
    Gibbs function     -1.5917e+06       -4.611e+07     J
 heat capacity c_p          995.12        2.883e+04     J/K
 heat capacity c_v          708.11        2.051e+04     J/K

                           X                 Y          Chem. Pot. / RT
                     -------------     ------------     ------------
                O2           0.21         0.231957         -27.9346
                N2           

C:\Users\Skiadr\Anaconda3\lib\site-packages\ipykernel_launcher.py:184: RuntimeWarning: divide by zero encountered in double_scalars



  gri30:

       temperature         328.412  K
          pressure         91969.2  Pa
           density         0.97574  kg/m^3
  mean mol. weight         28.9697  amu

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy           30435        8.817e+05     J
   internal energy          -63820       -1.849e+06     J
           entropy          6984.7        2.023e+05     J/K
    Gibbs function     -2.2634e+06       -6.557e+07     J
 heat capacity c_p          1006.1        2.915e+04     J/K
 heat capacity c_v          719.11        2.083e+04     J/K

                           X                 Y          Chem. Pot. / RT
                     -------------     ------------     ------------
                O2           0.21         0.231957         -26.3472
                N2           0.78         0.754253         -23.3943
                AR           0.01        0.0137896         -23.3234
     [  +50 minor]        


  gri30:

       temperature         294.716  K
          pressure         50553.7  Pa
           density        0.597554  kg/m^3
  mean mol. weight         28.9642  amu

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy          -20400       -5.909e+05     J
   internal energy       -1.05e+05       -3.041e+06     J
           entropy          7052.5        2.043e+05     J/K
    Gibbs function     -2.0989e+06       -6.079e+07     J
 heat capacity c_p            1003        2.905e+04     J/K
 heat capacity c_v          715.96        2.074e+04     J/K

                           X                 Y          Chem. Pot. / RT
                     -------------     ------------     ------------
                H2    6.35509e-09      4.42308e-10         -35.2868
                 O    4.19413e-12      2.31678e-12          55.4229
                O2       0.208705         0.230571          -26.936
                OH    9.80


  gri30:

       temperature         742.065  K
          pressure     1.37954e+06  Pa
           density         6.48408  kg/m^3
  mean mol. weight         28.9995  amu

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy      4.5542e+05        1.321e+07     J
   internal energy      2.4266e+05        7.037e+06     J
           entropy          7057.2        2.047e+05     J/K
    Gibbs function     -4.7815e+06       -1.387e+08     J
 heat capacity c_p          1089.5         3.16e+04     J/K
 heat capacity c_v          802.82        2.328e+04     J/K

                           X                 Y          Chem. Pot. / RT
                     -------------     ------------     ------------
                H2    3.81122e-13      2.64934e-14         -42.8028
                O2       0.209587         0.231264         -24.7717
                OH    9.42689e-14      5.52859e-14         -44.2202
               H2O    3.51

...

no(M)[%]


<Figure size 640x480 with 1 Axes>

n(M)[%]


<Figure size 640x480 with 1 Axes>


  gri30:

       temperature          166929  K
          pressure          166929  Pa
           density       0.0034756  kg/m^3
  mean mol. weight         28.8977  amu

                          1 kg            1 kmol
                       -----------      ------------
          enthalpy     -6.4597e+13       -1.867e+15     J
   internal energy     -6.4597e+13       -1.867e+15     J
           entropy     -4.8068e+08       -1.389e+10     J/K
    Gibbs function      1.5643e+13         4.52e+14     J
 heat capacity c_p     -1.9726e+09         -5.7e+10     J/K
 heat capacity c_v     -1.9726e+09         -5.7e+10     J/K

                           X                 Y          Chem. Pot. / RT
                     -------------     ------------     ------------
                H2    1.88557e-08      1.31536e-09          -710231
                 H    3.53951e-11      1.23457e-12         -50.5468
                 O    8.46789e-08      4.68829e-08         -44042.1
                O2       0